In [17]:
import cv2
import os
import numpy as np
import torch
from matplotlib import colors
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/siamese-registration')

from datasets import FolderFramesDataset
from models import *
from utils import get_transformation_matrix

In [4]:
path = "/content/drive/MyDrive/data"
output_path = "/content/drive/MyDrive/outputs"

In [5]:
test_dataset = FolderFramesDataset(
    transforms=transforms.Compose([
        transforms.ToTensor(),
    ]),
    path=os.path.join(path, "frame_sequences"),
)

In [6]:
test_loader = DataLoader(test_dataset, batch_size=1, num_workers=2, shuffle=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
checkpoint = "model-9.pt"
model = siamese_resnet18(1, 7)
model.load_state_dict(torch.load(os.path.join(output_path, checkpoint), map_location=device))
model.cuda()
model.eval()

print(f"Running on {device}")

Running on cuda


In [21]:
image_list = []
image_list_warped = []

with tqdm(test_loader) as validation_progress:
    for img0, img1 in validation_progress:
        

        if len(image_list) == 0:
            img0, img1 = img0.to(device=device), img1.to(device=device)
            outputs = model(img0, img1)
            image0 = img0.detach().cpu().numpy()[0, 0, :, :]
            image1 = img1.detach().cpu().numpy()[0, 0, :, :]

        else:
            img1 = img1.to(device=device)
            img3 = img0
            img3[0, 0, :, :] = torch.from_numpy(image_list_warped[-1])
            img3 = img3.to(device=device)
            outputs = model(img3, img1)
            image0 = img3.detach().cpu().numpy()[0, 0, :, :]
            image1 = img1.detach().cpu().numpy()[0, 0, :, :]
        
        rows, cols = image0.shape
        center = (cols//2, rows//2)
        tx, ty, sx, sy, shx, shy, q = outputs.detach().cpu().numpy().reshape(-1).tolist()
        matrix = get_transformation_matrix(center, tx, ty, sx, sy, shx, shy, q)
        matrix_opencv = np.float32(matrix.flatten()[:6].reshape(2, 3))
        inverse_matrix = cv2.invertAffineTransform(matrix_opencv)
        warped_image = cv2.warpAffine(image1, inverse_matrix, (cols, rows))

        if len(image_list) == 0:
            image_list.append(image0)
            image_list_warped.append(image1)
        else:
            image_list.append(image0)
            image_list_warped.append(warped_image)


100%|██████████| 251/251 [00:11<00:00, 21.24it/s]


In [22]:
img = np.concatenate((image_list[0], image_list_warped[0]), axis = 1)

# Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
out = cv2.VideoWriter('/content/drive/MyDrive/outputs/video/outpy2.mp4', cv2.VideoWriter_fourcc(*'XVID'), 15, (1500, 500), 0)

for image, image_warped in zip(image_list, image_list_warped):
    img = np.concatenate((image, image_warped), axis = 1)
    I = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    out.write(I)

cv2.destroyAllWindows()
out.release()
